# Deep Learning in Audio Classification in Python

## Predicting Sick and Non-Sick Audio

```
What's the Execution Plan?
- The data is in the directory Dataset
  - further in the directories: 'Train' 'Test' and 'Validation'
- Each Set has two directories named by the dataset classes

What's the dataset Size?
- Its Big !!!

Is it Big Data Problem?
- Yes

Do I have resources to use hadoop/aws?
- no, I'm in Lockdown and limited time and knowledge is a concern for me!!

What's the solution?
- Have to use my old Intel i3 core :/ laptop to do some basic template
- once I get internet access, I'll use the template to run on Google's Colab =')
- After debugging, I'll increase the full dataset and re-run the program files for visualizaton, model training :O
```

### Importing Libraries

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile

In [2]:
from python_speech_features import mfcc

In [3]:
from tqdm import tqdm

In [4]:
from keras.models import load_model

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) 

In [5]:
import pickle

In [6]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

### User Defined

In [7]:
def build_predictions(audio_dir):
    y_true = []
    y_pred = []
    
    fn_prob = {}
    
    print('Extracting features from audio')
    
    for fn in tqdm(os.listdir(audio_dir)):
        rate, wav = wavfile.read(os.path.join(audio_dir, fn))
        Class = fn2class[fn]
        c = classes.index(Class)
        y_prob = []
        
        for i in range(0,wav.shape[0] - config.step, config.step):
            sample = wav[i : i+config.step]
            x = mfcc(sample, rate, numcep=config.nfeat, nfilt=config.nfilt, nfft=config.nfft)
            
            x = (x - config.min) / (config.max - config.min)
            
            if config.mode == 'conv':
                x = x.reshape(1, x.shape[0], x.shape[1], 1)
            elif config.mode == 'time':
                x = np.expand_dims(x, axis = 0)
            
            y_hat = model.predict(x)
            y_prob.append(y_hat)
            y_pred.append(np.argmax(y_hat))
            y_true.append(c)
            
        fn_prob[fn] = np.mean(y_prob, axis = 0).flatten()
    return y_true, y_pred, fn_prob

In [20]:
class Config:
    def __init__(self, mode= 'conv', nfilt=26, nfeat=13, nfft = 2048, rate = 16000):
        self.mode = mode
        self.nfilt = nfilt
        self.nfeat = nfeat
        self.nfft = nfft
        self.rate = rate
        self.step = int(rate/10)
        self.model_path = os.path.join('models', mode + '.model')
        self.p_path = os.path.join('pickles', mode + '.p')

## Data Extraction

In [8]:
os.listdir('Temp_Dataset/')

['test', 'train', 'validation']

In [9]:
classes = list(os.listdir('Temp_Dataset/validation/'))

print("Number of Classes in the Data Set:", len(classes), "Classes")
print("The classes of the dataset are   :", classes[0], ",", classes[1])

Number of Classes in the Data Set: 2 Classes
The classes of the dataset are   : not_sick , sick


#### Creating the dataframe with basic column names

In [10]:
column_names = ['Fname','Class', 'Length']
df = pd.DataFrame(columns = column_names)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 3 columns):
Fname     0 non-null object
Class     0 non-null object
Length    0 non-null object
dtypes: object(3)
memory usage: 0.0+ bytes


In [11]:
# dataset_directory = 'Dataset/validation/'
dataset_directory = 'Temp_Dataset/train/'

In [12]:
for c in list(classes):
    print('Number of files in the directory \'{}\' are {}'.format(c,len(os.listdir(dataset_directory+c))))

Number of files in the directory 'not_sick' are 385
Number of files in the directory 'sick' are 313


In [13]:
for c in list(classes):
    for n,f in tqdm(enumerate(os.listdir(dataset_directory+c))):
        rate, signal = wavfile.read(dataset_directory+str(c)+"/"+str(f))
        length = signal.shape[0]/rate
        f_df = pd.DataFrame({
            "Fname": str(f),
            "Class": str(c),
            "Length": length}, index = [n])
        df = df.append(f_df)

385it [00:01, 333.43it/s]
313it [00:00, 340.42it/s]


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 698 entries, 0 to 312
Data columns (total 3 columns):
Fname     698 non-null object
Class     698 non-null object
Length    698 non-null float64
dtypes: float64(1), object(2)
memory usage: 21.8+ KB


In [15]:
fn2class = dict(zip(df.Fname, df.Class))

In [16]:
len(fn2class)

698

# Loading Models

In [21]:
p_path = os.path.join('pickles', 'time.p')

In [22]:
with open(p_path, 'rb') as handle:
    config = pickle.load(handle)

In [23]:
model = load_model(config.model_path)

# Prediction

In [25]:
y_true, y_pred, fn_prob = build_predictions('Temp_train')

  0%|                                                  | 0/698 [00:00<?, ?it/s]

Extracting features from audio


100%|████████████████████████████████████████| 698/698 [11:30<00:00,  1.01it/s]


In [26]:
acc_score = accuracy_score(y_true=y_true, y_pred=y_pred)
acc_score

0.7901531728665208

In [27]:
print(classification_report(y_true=y_true, y_pred=y_pred))

             precision    recall  f1-score   support

          0       0.90      0.70      0.79     52936
          1       0.71      0.91      0.79     43034

avg / total       0.81      0.79      0.79     95970



             precision    recall  f1-score   support

          0       0.85      0.79      0.82     43035
          1       0.75      0.81      0.78     33129

avg / total       0.80      0.80      0.80     76164



In [28]:
y_probs = []

for i, row in df.iterrows():
    y_prob = fn_prob[row.Fname]
    y_probs.append(y_prob)
    for c, p in zip(classes, y_prob):
        df.at[i,c] = p

In [29]:
y_predictions = [classes[np.argmax(y)] for y in y_probs]
df['y_pred'] = y_predictions

In [30]:
df.to_csv('predictions.csv', index = False)